# Lecture 14 - Uncertainty Propagation (Sampling Methods II)

## Objectives

+ to use variance reduction techniques to accelerate MC
+ to use importance sampling to accelerate MC
+ to use latin hyper-cube designs to accelerate MC in moderate dimensions
+ to propagate uncertainty through an initial value problem

## Readings

+ Monte Carlo Strategies in Scientific Computing (Jun S. Liu, 2001): Chapters 1 and 2.

+ Monte Carlo Statistical Methods (Christian P. Robert and George Casella, 1999): Chapter 3.

+ McKay, M.D.; Beckman, R.J.; Conover, W.J. (May 1979). A Comparison of Three Methods for Selecting Values of Input Variables in the Analysis of Output from a Computer Code. Technometrics (JSTOR Abstract) (American Statistical Association) 21 (2): 239–245. doi:10.2307/1268522. ISSN 0040-1706. JSTOR 1268522. OSTI 5236110

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

## Importance Sampling
Let $X$ be a random variable with probability density $p(x)$ and $f(x)$ be a function.
As in the lecture 13, we wish to estimate the expectation:
$$
I = \mathbb{E}_p[f(X)]=\int f(x)p(x)dx.
$$
Note that we have explicitly marked the expectation operator with $p$.
This is because in a while we will change $p$ to something else.

The idea in *importance sampling* is to re-write $I$ as an expectation with respect to another probability density, say $q(x)$.
The only requirement is that the support of $q(x)$ must be broader than the support of $p(x)$ ($q(x)$ must be positive wherever $p(x)$ is positive).
Here we go:
$$
I = \mathbb{E}_p[f(X)] = \int f(x)p(x)dx = \int \frac{f(x)p(x)}{q(x)}q(x)dx = \mathbb{E}_q\left[\frac{f(x)p(x)}{q(x)}\right].
$$
This suggests that you can estimate $I$ by sampling from $q(x)$ instead of $p(x)$.
A trivial estimator is:
$$
I_{n} = \frac{1}{n}\sum_{j=1}^n\frac{f(X_j)p(X_j)}{q(X_j)}, 
$$
where $X_1,\dots,X_n\sim q$ iid.

Why on earth is this useful?

+ First, it may be much easier to sample from $q(x)$ than it is to sample from $p(x)$.
+ You can reuse samples from $q(x)$ with different $f(x)$ and different $p(x)$.
+ With the right choice of $q(x)$ you can actually reduce the variance of the MC estimator.

Let's see how it works in practice.

### Example: Small Tail Probabilities (Casella 3.11)
Let $X\sim \mathcal{N}(0,1)$ and assume that we want to compute:
$$
I = \mathbb{P}[X > 4.5].
$$
The standard MC estimator is:
$$
I_n = \frac{1}{n}\sum_{j=1}^n1_{[4.5,+\infty]}(X_j),
$$
for $X_j\sim \mathcal{N}(0,1)$ iid.
Let's see how it does.

In [ ]:
num_mc = 10000
s = 0.
s2 = 0.
for i in range(num_mc):
    x = np.random.randn()
    if x > 4.5:
        s += 1.
        s2 += 1.
I_n = s / num_mc
V_n = (s2 / num_mc - I_n ** 2) / num_mc
print('I_n = %1.2e +- %1.2e' % (I_n, 2 * np.sqrt(V_n)))

It doesn't look right for $n=10,000$. We did not count a single event. You would have to go to $n=1,000,000$ to hit one or two events.
To get an accurate answer you probably need hundrends of millions of samples.

Let's see if we can remedy this situtation with importance sampling.
Ideally, we only want to sample $X$'s that are greater than $4.5$.
Let us introduce a random variable $Y\sim \mathcal{TE}(4.5, 1)$ following an exponential distribution with rate parameter $1$ which is truncated at $4.5$.
The density of $Y$ is:
$$
q(y) = \frac{e^{-y}}{c}1_{[4.5,+\infty)}(y),
$$
where $c$ is a normalization constant.
To find the normalization constant we impose:
$$
\int q(y) dy = 1 \Rightarrow c = \int_0^{+\infty}e^{-y}1_{[4.5,+\infty)}(y)dy = \int_{4.5}^{+\infty}e^{-y}dy = \left[-e^{-y}\right]_{4.5}^{+\infty}=e^{-4.5}.
$$
So, the density becomes:
$$
q(y) = e^{-(y-4.5)}1_{[4.5,+\infty)}(y).
$$
Let's visualize it comparing it to the standard normal.

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(-4., 10., 200)
p = st.norm.pdf(x)
q = np.exp(-(x-4.5))
q[x < 4.5] = 0.
ax.plot(x, p, label='$p(x)$')
ax.plot(x, q, label='$q(x)$')
ax.set_xlabel('$x$')
ax.set_ylabel('Probability density')
plt.legend(loc='best');

The importance sampling estimator is:
$$
I_{2,n} = \frac{1}{n}\sum_{j=1}^n\frac{f(X_j)p(X_j)}{q(X_j)}=\frac{1}{n\sqrt{2\pi}}\sum_{j=1}^ne^{-\frac{1}{2}X_j^2+X_j-4.5},
$$
for $X_1,\dots,X_n\sim q$ iid.
Let's see what we are going to get:

In [ ]:
num_mc = 10000
s = 0.
s2 = 0.
for i in range(num_mc):
    x = np.random.exponential() + 4.5 # Just add 4.5 to a standard exponential
    f = np.exp(-0.5 * x ** 2 + x - 4.5) / np.sqrt(2. * np.pi)
    s += f
    s2 += f ** 2
I_n = s / num_mc
V_n = (s2 / num_mc - I_n ** 2) / num_mc
print('I_n = %1.4e +- %1.4e' % (I_n, 2*np.sqrt(V_n)))

### Finite variance concerns
The variance of an importance sampling estimator is:
$$
V_n = \mathbb{V}[I_n] = \frac{1}{n}\mathbb{V}_q\left[\frac{f(X_j)p(X_j)}{q(X_j)}\right]=\frac{1}{n}\left(\mathbb{E}_q\left[\frac{f^2(X_j)p^2(X_j)}{q^2(X_j)}\right]-\left(\mathbb{E}_q\left[\frac{f(X_j)p(X_j)}{q(X_j)}\right]\right)^2\right).
$$
So, in order to have a finite variance estimator, we must have that:
$$
\mathbb{E}_q\left[\frac{f^2(X_j)p^2(X_j)}{q^2(X_j)}\right] = \int f^2(x)\frac{p^2(x)}{q(x)}dx < \infty.
$$
Unless, this condition is satisfied, you will get garbage.

### Example: Infinite Variance Estimator.
It is absolutely crucial that the variance of your estimator is finite.
Otherwise, you cannot trust it even if it theoretically converges.
Let us demonstrate this by constructing a really stupid importance sampling desnity.
Let $X\sim\mathcal{E}(1)$ (exponential with unit rate) so that the density is:
$$
p(x) = e^{-x}_{[0,\infty)}(x).
$$
Pick $f(x)=x$ so that we want to compute the mean $\mathbb{E}[f(X)] = \mathbb{E}_p[X]=1$ (analytically known).
As importance sampling density let's use the density of a $\mathcal{N}(0,1)$:
$$
q(x) = \frac{1}{\sqrt{2\pi}}e^{-\frac{1}{2}x^2}.
$$
Let's look at that integral that dominates the variance:
$$
\int f^2(x)\frac{p^2(x)}{q(x)}dx = \int_0^\infty x^2 \frac{e^{-2x}}{\frac{1}{\sqrt{2\pi}}e^{-\frac{1}{2}x^2}}dx = \infty.
$$
Let's implement an MC estimate using this importance sampling density and see what we get.

In [ ]:
# How many times to try
num_tries = 10
# How many MC steps to take per try 
num_mc = 1000
# For plotting purposes
fig, ax = plt.subplots()
for i in range(num_tries):
    I = np.ndarray((num_mc, ))
    s = 0.
    for j in range(num_mc):
        x = np.random.randn()
        if x >= 0.:
            y = x * np.exp(-x) / st.norm.pdf(x)
            s += y
        I[j] = s / (j + 1)
    plt.plot(np.arange(1, num_mc + 1), I, color=sns.color_palette()[0], lw=1)
ax.set_xlabel('$n$')
ax.set_ylabel('$I_n$')
plt.plot(np.arange(1, num_mc + 1), np.ones((num_mc, )), color='r');

### Questions

+ Do a single trial for a large number of samples. Does it converge? Remember, that in reality you never know the truth (red line). If you think that you have found a number of samples that guarantees convergence, run it once more. Does it converge now?

+ Increase the number of tries to 10, 100, 1000 (decrease the number of samples per trial if it becomes to slow). What do you observe?

### The Best Importance Sampling Distribution
So, the choice of $q(x)$ is completely up to us.
What if we picked it so that we minimize the variance of the estimator?
That is, we would like to find the $q^*(x)$ that solves minimizes:
$$
\min_{q}\int f^2(x)\frac{p^2(x)}{q(x)}dx.
$$
We will do it by taking the derivative of the functional:
$$
J[q] = \int f^2(x)\frac{p^2(x)}{q(x)}dx,
$$
with respect to an arbitrary direction and setting it equal to zero.
To this end, let $\eta(x)$ be any non-negative function such that
$$
\int \eta(x)dx = 0.
$$
Then, for any $\epsilon>0$, we have that the pertubation $q(x) + \epsilon\eta(x)$ is a valid probability density since:
$$
\int (q(x) + \epsilon\eta(x)) dx = \int q(x) dx + \epsilon\int\eta(x)dx = 1.
$$
The first variation of $J[q]$ in the direction of $\eta(x)$ is:
$$
\begin{array}[ccc]
\delta J[q]/\delta \eta &=& \lim_{\epsilon\rightarrow 0}\frac{J[q + \epsilon\eta]-J[q]}{\epsilon}\\
&=& \lim_{\epsilon\rightarrow 0}\frac{\int \left[f^2(x)\frac{p^2(x)}{q(x)+\epsilon\eta(x)}-f^2(x)\frac{p^2(x)}{q(x)}\right]dx}{\epsilon}\\
&=& \lim_{\epsilon\rightarrow 0}\frac{\int f^2(x)p^2(x)\left[\frac{1}{q(x)+\epsilon\eta(x)}-\frac{1}{q(x)}\right]dx}{\epsilon}\\
&=& \lim_{\epsilon\rightarrow 0}\frac{\int f^2(x)p^2(x)\left[\frac{q(x)-q(x)-\epsilon\eta(x)}{\left(q(x)+\epsilon\eta(x)\right)q(x)}\right]dx}{\epsilon}\\
&=&-\lim_{\epsilon\rightarrow 0}\int f^2(x)p^2(x)\left[\frac{\eta(x)}{\left(q(x)+\epsilon\eta(x)\right)q(x)}\right]dx\\
&=&-\int f^2(x)p^2(x)\left[\frac{\eta(x)}{q^2(x)}\right]dx.
\end{array}
$$
For an extremum, the first variation must be equal to zero (necessary condition) for all possible $\eta$'s.
We see that this is trivially satisfied for:
$$
q^*(x) \propto |f(x)|p(x).
$$
To show that the above defined $q^*$ is a minimum, we need to take the second variation of $J[q]$ and show that it is positive for all $\eta$'s.
We leave this as an excersise.

### Questions

+ Is it really possible to use the minimum variance importance sampling density? 

## Latin Hyper-Cube Designs

Latin hyper-cube designs (LHS) are quasi-random sequences that resemble uniform random nummbers, but have better convergence properties than truly random numbers.

+ A **latin square** is a square grid containing samples only one sample in each row and each column.
+ A **latin hypercube** is the generalisation of this concept to many dimensions.

Here is how they look like in 2D.

In [ ]:
import design # Library the implements latin hyper-cube designs you must install it: pip install py-design
fig, ax = plt.subplots()
ax.grid(which='major')
for i in range(1):
    X = design.latin_center(5, 2)
    ax.scatter(X[:,0], X[:,1], 50., color=sns.color_palette()[i])
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$');

Let's compare to uniform samples in 2D:

In [ ]:
num_samples = 5
fig, ax = plt.subplots()
X_lhs = design.latin_center(num_samples, 2)
ax.scatter(X_lhs[:,0], X_lhs[:,1], 50., color=sns.color_palette()[0])
X_lhs = design.latin_center(num_samples, 2)
ax.grid(which='major')
ax.scatter(X_lhs[:,0], X_lhs[:,1], 50., color=sns.color_palette()[2])
ax.set_title('Latin hypercube samples')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$');
fig2, ax2 = plt.subplots()
X_unif = np.random.rand(num_samples, 2)
ax2.grid(which='major')
ax2.scatter(X_unif[:,0], X_unif[:,1], 50., color=sns.color_palette()[1])
X_unif = np.random.rand(num_samples, 2)
ax2.scatter(X_unif[:,0], X_unif[:,1], 50., color=sns.color_palette()[3])
ax2.set_title('Uniform samples')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.set_xlabel('$x_1$')
ax2.set_ylabel('$x_2$');

## Uniform Sampling vs Latin Hyper-cubes
Consider the ODE:
  \begin{align*}
    \dot{y} = \frac{d y(t)}{dt} & =-ay(t),\qquad y(0) = y_0.
  \end{align*}
The solution is: $y(t) = Ie^{-at}$.
Let's make $a$ and $I$ random and use MC and LHS to find the mean, $E[y(t)]$, and the variance $V[y(t)]$.
We'll take:
$$
a \sim \mathcal{U}(0, 0.1),
$$
and
$$
y_0 \sim \mathcal{U}(8, 10).
$$
For convenience, let's map these random variables to standardized uniform random variables:
$$
x_i \sim \mathcal{U}(0, 1), i=1,2.
$$
This can be done by defining:
$$
a = 0.1 x_1.
$$
Using the new random variables, the ODE can be written as:
$$
\dot{y} = -0.1 x_1 y, y(0) = 8 + 2x_2.
$$

### The "Solver" object
Let's develop a solver for this problem. We'll make the solver work as a nice function that accepts a vector $\mathbf{x} = (x_1, x_2)$ as an input, and returns the solution of the ODE on a finite set of timesteps $0=t_1<t_2<\dots<t_{n_t}=T$. The solver, will make use of the functionality of [scipy.integrate.odeint](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.odeint.html). This is a trivial problem, but studying its structure will teach you a few useful Python tricks.

In [ ]:
import scipy.integrate

class Ex1Solver(object):
    """
    An object that can solver the afforementioned ODE problem.
    It will work just like a multivariate function.
    """
    
    def __init__(self, nt=100, T=5):
        """
        This is the initializer of the class.
        
        Arguments:
            nt - The number of timesteps.
            T  - The final time.
        """
        self.nt = nt
        self.T = T
        self.t = np.linspace(0, T, nt) # The timesteps on which we will get the solution
        # The following are not essential, but they are convenient
        self.num_input = 2             # The number of inputs the class accepts
        self.num_output = nt           # The number of outputs the class returns
    
    def __call__(self, x):
        """
        This special class method emulates a function call.
        
        Arguments:
            x - A 1D numpy array with 2 elements. This represents the stochastic input x = (x1, x2).
        """
        def rhs(y, t, x1):
            """
            This is the right hand side of the ODE.
            """
            return -.1 * x1 * y
        # The initial condition
        y0 = [8 + 2 * x[1]]
        # We are ready to solve the ODE
        y = scipy.integrate.odeint(rhs, y0, self.t, args=(x[0],)).flatten()
        # The only strange thing here is the use of ``args`` to pass they x1 argument to the rhs().
        # That's it
        return y

Since this is the first time we see a Python class, let's play with it.

In [ ]:
# Here is how you can initialize it:
solver = Ex1Solver()
# You can access its arguments:
print "Number of timesteps:", solver.nt
print "Final time:", solver.T
print "Timesteps:", solver.t
print "Num inputs:", solver.num_input
print "Num outputs:", solver.num_output

In [ ]:
# If you wish, you may intiialize it with a different initial arguments:
solver = Ex1Solver(nt=200, T=50)
print "Number of timesteps:", solver.nt
print "Final time:", solver.T
print "Timesteps:", solver.t
print "Num inputs:", solver.num_input
print "Num outputs:", solver.num_output

In [ ]:
# Now let's evaluate the solver at a specific input.
# You can just use it as a function
x = [0.5, 0.5]
y = solver(x)
print y

In [ ]:
# Let's plot it:
fig, ax = plt.subplots()
ax.plot(solver.t, y)
ax.set_xlabel('$t$')
ax.set_ylabel('$y(t)$')

In [ ]:
# Now, let's just plot a few random samples.
fig, ax = plt.subplots()
for i in xrange(10):
    x = np.random.rand(2)
    y = solver(x)
    plt.plot(solver.t, y)
ax.set_xlabel('$t$')
ax.set_ylabel('$y(t)$');

### Propagating Uncertainties with MC

Let's propagate the uncertainties through the ODE using MC.

In [ ]:
# Let's now do Monte Carlo to compute the mean and the variance
# This is to accumulate the sum of all outputs
y_mc = np.zeros(solver.num_output)
# This is to accumlate the square of all outputs
y2_mc = np.zeros(solver.num_output)
# Pick the number of samples you wish to do:
num_samples = 10000
# Let's do it
data_mc = []
for i in xrange(num_samples):
    if i % 1000 == 0:
        print 'sample', i + 1, 'from', num_samples
    x = np.random.rand(2)
    y = solver(x)
    y_mc += y
    y2_mc += y ** 2
    data_mc.append(y)
data_mc = np.array(data_mc)
# Now we are ready for the mean estimate:
y_m_mc = y_mc / num_samples
# And the variance estimate
y_v_mc = y2_mc / num_samples - y_m_mc ** 2

In [ ]:
# Let's plot the mean
fig, ax = plt.subplots()
ax.plot(solver.t, y_m_mc)
ax.set_xlabel('$t$')
ax.set_ylabel('$\mathbb{E}[y(t)]$')

In [ ]:
# Let's plot the variance
fig, ax = plt.subplots()
ax.plot(solver.t, y_v_mc)
ax.set_xlabel('$t$')
ax.set_ylabel('$\mathbb{V}[y(t)]$')

In [ ]:
# Now, let's draw the predictive envelop
# We need the standard deviation:
y_s_mc = np.sqrt(y_v_mc)
# A lower bound for the prediction
y_l_mc = np.percentile(data_mc, 2.75, axis=0)
# An upper bound for the prediction
y_u_mc = np.percentile(data_mc, 97.5, axis=0)
# And let's plot it:
fig, ax = plt.subplots()
ax.plot(solver.t, y_m_mc)
ax.fill_between(solver.t, y_l_mc, y_u_mc, alpha=0.25)

### Propagating Uncertainties with LHS

Let's propagate the uncertainties through the ODE using LHS.
All we have to do is to copy and paste the above code and make some modifications.

In [ ]:
# Let's now do LHS to compute the mean and the variance
# This is to accumulate the sum of all outputs
y_lhs = np.zeros(solver.num_output)
# This is to accumlate the square of all outputs
y2_lhs = np.zeros(solver.num_output)
# Pick the number of samples you wish to do:
num_samples = 10000
# You have to create the LHS design prior to looping:
X = design.latin_center(num_samples, 2)
# Let's do it
data_lhs = []
for i in xrange(num_samples):
    if i % 1000 == 0:
        print 'sample', i + 1, 'from', num_samples
    x = X[i, :]
    y = solver(x)
    y_lhs += y
    y2_lhs += y ** 2
    data_lhs.append(y)
data_lhs = np.array(data_lhs)
# Now we are ready for the mean estimate:
y_m_lhs = y_lhs / num_samples
# And the variance estimate
y_v_lhs = y2_lhs / num_samples - y_m_lhs ** 2

In [ ]:
# Let's plot the mean
plt.plot(solver.t, y_m_lhs)
plt.xlabel('$t$')
plt.ylabel('$\mathbb{E}[y(t)]$')

In [ ]:
# Let's plot the variance
plt.plot(solver.t, y_v_lhs)
plt.xlabel('$t$')
plt.ylabel('$\mathbb{V}[y(t)]$')

In [ ]:
# Now, let's draw the predictive envelop
# We need the standard deviation:
y_s_lhs = np.sqrt(y_v_lhs)
y_l_lhs = np.percentile(data_lhs, 2.75, axis=0)
# An upper bound for the prediction
y_u_lhs = np.percentile(data_lhs, 97.5, axis=0)
# And let's plot it:
plt.plot(solver.t, y_m_lhs)
plt.fill_between(solver.t, y_l_lhs, y_u_lhs, alpha=0.25)

## Comparing MC to LHS

So, which one is better? MC or LHS? To test this, we need to establish a ground truth.
Fortunately, we can get an analytic solution to:
$$
\dot{y} = -0.1 x_1 y, y(0) = 8 + 2x_2.
$$
It is:
$$
y(t;x_1,x_2) = (8 + 2x_2)e^{-0.1 x_1 t}.
$$
We can integrate $x_1$ and $x_2$ analytically from this expression.

Doing so, we get the following mean solution and variance:
$$
\mu = \mathbb{E}[y] = \frac{90}{t} (1 - \exp^{-0.1t}), \\
\mathrm{S} = \mathbb{V}[y] =  \frac{2440}{6t}(1 - \exp^{-0.2t}) - \mu^2
$$

In [ ]:
def E_y(t):
    if t == 0:
        return 9.
    return (90*(1-np.exp(-0.1*t))) / t
E_y =  np.vectorize(E_y)

def V_y(t):
    if t == 0:
        return (2440*0.2)/6. - E_y(t) **2
    
    return (2440.*(1-np.exp(-0.2*t)))/(6.*t) - E_y(t)**2
V_y = np.vectorize(V_y)

In [ ]:
# Let's start by doing some plots that compare the variance obtained by MC and LHS to the true one
plt.plot(solver.t, V_y(solver.t), label='True variance')
plt.plot(solver.t, y_v_mc, '--', label='MC (10,000)')
plt.plot(solver.t, y_v_lhs, '-.', label='LHS (10,000))')
plt.legend(loc='best')

They seem close. But this is for 10,000 samples.
Let's test the convergence of each method. To do this, we will compute the evolution of the root square error in the variance:
$$
\mbox{RSE}_\alpha(n) = \left(\sum_{i=1}^{n_t}\left(V[y(t_i)] - V_{\alpha,n}[y(t_i)]\right)^2\right)^{\frac{1}{2}},
$$
where $\alpha = \{\mbox{MC}, \mbox{LHS}\}$ stands for the method, $n$ for the number of samples used, and $V_{\alpha,n}[y(t_i)]$ for the estimate of the variance of $y(t_i)$.
It will probably take a while to develop the code.

In [ ]:
def get_MC_rse(max_num_samples=100):
    """
    Get the maximum error of MC.
    """
    y_v_true = V_y(solver.t)
    y = np.zeros(solver.num_output)
    y2 = np.zeros(solver.num_output)
    n = []
    rse = []
    for i in xrange(max_num_samples):
        x = np.random.rand(2)
        y_sample = solver(x)
        y += y_sample
        y2 += y_sample ** 2
        if i % 1 == 0:    # Produce estimate every 100 steps
            n.append(i + 1)
            y_m = y / (i + 1)
            y_v = y2 / (i + 1) - y_m ** 2
            rse.append(np.linalg.norm(y_v_true - y_v))
    return n, rse

def _get_LHS_rse(max_num_samples=100):
    y_v_true = V_y(solver.t)
    y = np.zeros(solver.num_output)
    y2 = np.zeros(solver.num_output)
    n = []
    rse = []
    X = design.latin_center(max_num_samples, 2)
    for i in xrange(max_num_samples):
        x = X[i, :]
        y_sample = solver(x)
        y += y_sample
        y2 += y_sample ** 2
        if i % 1 == 0:    # Produce estimate every 100 steps
            n.append(i + 1)
            y_m = y / (i + 1)
            y_v = y2 / (i + 1) - y_m ** 2
            rse.append(np.linalg.norm(y_v_true - y_v))
    return n, rse

def get_LHS_rse(max_num_samples=100):
    n = []
    rse = []
    for i in xrange(max_num_samples):
        if i % 1 == 0:
            _n, _rse = _get_LHS_rse(i + 1)
            n.append(_n[-1])
            rse.append(_rse[-1])
    return n, rse

In [ ]:
n_mc, rse_mc = get_MC_rse(50)
n_lhs, rse_lhs = get_LHS_rse(50)
plt.plot(n_mc, rse_mc, label='MC RSE')
plt.plot(n_lhs, rse_lhs, label='LHS RSE')
plt.xlabel('$n$')
plt.ylabel('RSE')
plt.legend(loc='best')

It is hard to tell which one is better...
There is a considerable ammount of epistemic uncertainty.
How would you go about quantifying it?
Notice, that everytime you run the above code, it produces a different estimate. This suggests, that if you repeat the procedure, say 100 times, you will be able to get predictive error bars for the error!

In [ ]:
num_exper = 1
num_samples = 20
rse_mc_samples = []
rse_lhs_samples = []
for i in xrange(num_exper):
    print i
    n_mc, rse_mc = get_MC_rse(num_samples)
    n_lhs, rse_lhs = get_LHS_rse(num_samples)
    rse_mc_samples.append(rse_mc)
    rse_lhs_samples.append(rse_lhs)
rse_mc_samples = np.array(rse_mc_samples)
rse_lhs_samples = np.array(rse_lhs_samples)
rse_mc_m = np.mean(rse_mc_samples, axis=0)
rse_mc_l = np.percentile(rse_mc_samples, 2.75, axis=0)
rse_mc_u = np.percentile(rse_mc_samples, 97.5, axis=0)
rse_lhs_m = np.mean(rse_lhs_samples, axis=0)
rse_lhs_l = np.percentile(rse_lhs_samples, 2.75, axis=0)
rse_lhs_u = np.percentile(rse_lhs_samples, 97.5, axis=0)
plt.plot(n_mc, rse_mc_m, label='MC')
plt.plot(n_lhs, rse_lhs_m, '--', label='LHS')
plt.fill_between(n_mc, rse_mc_l, rse_mc_u, alpha=0.25)
plt.fill_between(n_lhs, rse_lhs_l, rse_lhs_u, color='g', alpha=0.25)
plt.legend(loc='best')
plt.xlabel('$n$')
plt.ylabel('RSE')

### Questions

+ To get a more accurate picture of the convergence rate of the two methods increaes the number of trials ``num_expr``. You will have to wait...

+ LHS seems to perform better. Does it have a different convergence rate than MC?